In [ ]:
import numpy as np
import pandas as pd

selected_sensors_df = pd.read_csv("../data/selected_sensors2_cleaned.csv", index_col=0)

In [ ]:
selected_sensors_df

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error

## Prophet winter overfit 

In [ ]:
def train_test_split(series, test_size=0.2):
    """
    Splits the series into train and test sets.

    Parameters:
        series (array-like): The transformed series to split.
        test_size (float): Proportion of the series to include in the test set (default: 0.2).
    
    Returns:
        tuple: (train_series, test_series)
    """
    try:
        n = len(series)
        test_count = int(n * test_size)
        train_series = series[:-test_count]
        test_series = series[-test_count:]
        return train_series, test_series
    except Exception as e:
        warnings.warn(f"Failed to split series into train and test sets: {e}")
        return series, None


In [ ]:
# Install Prophet if not already installed
# !pip install prophet

from prophet import Prophet
import pandas as pd
import matplotlib.pyplot as plt

# Load and prepare data for Prophet
df_prophet = df_train.reset_index()
df_prophet.rename(columns={'date_time': 'ds', 'value': 'y'}, inplace=True)

# Define the Prophet model
model = Prophet(seasonality_mode='multiplicative', yearly_seasonality=True, weekly_seasonality=True, 
                holidays=kazakhstan_holidays
                )
model.add_seasonality(name='monthly', period=30.5, fourier_order=8)

In [ ]:
from itertools import product

# Define parameter grid
param_grid = {
    'changepoint_prior_scale': np.arange(0.01, 0.25, 0.05),
    'seasonality_prior_scale': np.arange(0.5, 5, 0.5)
}

# Run a grid search
best_mape = float('inf')
best_params = None

for params in product(param_grid['changepoint_prior_scale'], param_grid['seasonality_prior_scale']):
    model = Prophet(changepoint_prior_scale=params[0], seasonality_prior_scale=params[1], yearly_seasonality=True, weekly_seasonality=True, seasonality_mode='multiplicative')
    model.fit(df_prophet)
    forecast = model.predict(future)
    
    # Evaluate
    forecast_values = forecast[['ds', 'yhat']].set_index('ds').loc[test_data_prophet['ds']]
    mape = mean_absolute_percentage_error(test_data_prophet['y'], forecast_values['yhat']) * 100
    
    if mape < best_mape:
        best_mape = mape
        best_params = params

print(f"Best Params: {best_params}, Best MAPE: {best_mape}%")


## lstm